In [1]:
import csv
import json

import mindspore
from mindspore.dataset import transforms
from mindspore import nn

from mindnlp.dataset import load_dataset

from mindnlp.engine import Trainer, Evaluator
from mindnlp.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp.metrics import Accuracy

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

In [2]:
mindspore.set_seed(42)

In [3]:
# 下载tsv格式wrime数据集
!mkdir -p ./wrime_data
!wget -P ./wrime_data https://raw.githubusercontent.com/ids-cv/wrime/master/wrime-ver2.tsv

--2024-12-24 12:58:26--  https://raw.githubusercontent.com/ids-cv/wrime/master/wrime-ver2.tsv
Resolving proxy-notebook.modelarts.com (proxy-notebook.modelarts.com)... 192.168.0.33
Connecting to proxy-notebook.modelarts.com (proxy-notebook.modelarts.com)|192.168.0.33|:8083... connected.
Proxy request sent, awaiting response... 200 OK
Length: 8182156 (7.8M) [text/plain]
Saving to: ‘./wrime_data/wrime-ver2.tsv’

wrime-ver2.tsv      100%[===================>]   7.80M   718KB/s    in 11s     

2024-12-24 12:58:37 (748 KB/s) - ‘./wrime_data/wrime-ver2.tsv’ saved [8182156/8182156]



In [4]:
# 将TSV数据集文件进行划分并转换为JSON文件

# 定义保存路径
_OUTPUT_DIR = "./wrime_data"  # 保存JSON文件的目录

# 定义输出文件名
TRAIN_JSON = f"{_OUTPUT_DIR}/train.json"
VAL_JSON = f"{_OUTPUT_DIR}/val.json"
TEST_JSON = f"{_OUTPUT_DIR}/test.json"


# 将TSV文件转换为JSON文件
def tsv_to_json(tsv_file, train_json, val_json, test_json, remove_neutral=True):
    train_data = []
    val_data = []
    test_data = []

    with open(tsv_file, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter='\t')
        for row in reader:
            sentiment_score = int(row["Avg. Readers_Sentiment"])
            label = None

            if sentiment_score > 0:
                label = "positive"
                label = 0
            elif sentiment_score < 0:
                label = "negative"
                label = 1
            else:
                label = "neutral"
                label = 2

            if remove_neutral and label == 2:
                continue

            data_point = {
                "sentence": row["Sentence"],
                "label": label,
                # "user_id": int(row["UserID"]),
                # "datetime": row["Datetime"].strip()
            }

            # 根据 Train/Dev/Test 列划分数据集
            if row["Train/Dev/Test"].lower() == "train":
                train_data.append(data_point)
            elif row["Train/Dev/Test"].lower() == "dev":
                val_data.append(data_point)
            elif row["Train/Dev/Test"].lower() == "test":
                test_data.append(data_point)


    # 保存为JSON文件
    with open(train_json, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, ensure_ascii=False, indent=4)
        print(f"Saved train data to {train_json}, size = {len(train_data)}")

    with open(val_json, 'w', encoding='utf-8') as f:
        json.dump(val_data, f, ensure_ascii=False, indent=4)
        print(f"Saved validation data to {val_json}, size = {len(val_data)}")

    with open(test_json, 'w', encoding='utf-8') as f:
        json.dump(test_data, f, ensure_ascii=False, indent=4)
        print(f"Saved test data to {test_json}, size = {len(test_data)}")

if __name__ == "__main__":
    # 将TSV文件转换为JSON文件
    tsv_file = f"{_OUTPUT_DIR}/wrime-ver2.tsv"
    tsv_to_json(tsv_file, TRAIN_JSON, VAL_JSON, TEST_JSON, remove_neutral=True)

Saved train data to ./wrime_data/train.json, size = 20149
Saved validation data to ./wrime_data/val.json, size = 1608
Saved test data to ./wrime_data/test.json, size = 1781


In [5]:
# 加载训练集和验证集
wrime_train = load_dataset("json", data_files={"train": "./wrime_data/train.json"})
wrime_val = load_dataset("json", data_files={"validation": "./wrime_data/val.json"})

Generating train split: 20149 examples [00:00, 138648.14 examples/s]
Generating validation split: 1608 examples [00:00, 91531.96 examples/s]


In [6]:
def show_dataset_info(dataset):
    print("dataset column: {}".format(dataset.get_col_names()))
    print("dataset size: {}".format(dataset.get_dataset_size()))
    print("dataset batch size: {}\n".format(dataset.get_batch_size()))
show_dataset_info(wrime_train)
show_dataset_info(wrime_val)

dataset column: ['sentence', 'label']
dataset size: 20149
dataset batch size: 1

dataset column: ['sentence', 'label']
dataset size: 1608
dataset batch size: 1



In [7]:
from mindnlp.transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('tohoku-nlp/bert-base-japanese-v3')
test_tokenized = tokenizer('世界')
print(test_tokenized.keys())
print(test_tokenized.values())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
dict_values([[2, 12575, 3], [0, 0, 0], [1, 1, 1]])


In [8]:
import numpy as np

def process_dataset(dataset, tokenizer, max_seq_len=256, batch_size=32, shuffle=False):

    is_ascend = mindspore.get_context('device_target') == 'Ascend'
    def tokenize(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['token_type_ids'], tokenized['attention_mask']

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="sentence", output_columns=['input_ids', 'token_type_ids', 'attention_mask'])   
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                             'token_type_ids': (None, 0),
                                                             'attention_mask': (None, 0)})

    return dataset

In [9]:
dataset_train = process_dataset(wrime_train, tokenizer, shuffle=True)
dataset_val = process_dataset(wrime_val, tokenizer)

In [14]:
# 动态学习率计算
def dynamic_lr(lr, total_step, warmup_ratio=0.1):
    """
    计算线性学习率调度器的学习率列表。

    参数:
    lr (float): 初始学习率
    total_step (int): 总的训练步骤数
    warmup_ratio (float): warmup 阶段的比例，默认为 0.1

    返回:
    lrs (list): 学习率列表
    """
    lrs = []
    warmup_steps = int(total_step * warmup_ratio)  # 计算 warmup 步数
    decay_steps = total_step - warmup_steps  # 计算衰减阶段的步数

    # Warmup 阶段：学习率从 0 线性增加到初始学习率
    for i in range(1, warmup_steps+1):
        warmup_factor = float(i) / max(1, warmup_steps)
        lrs.append(lr * warmup_factor)

    # 衰减阶段：学习率从初始学习率线性衰减到 0
    for i in range(1, decay_steps+1):
        decay_factor = 1.0 - float(i) / max(1, decay_steps)
        lrs.append(lr * decay_factor)

    return lrs

# 数据集大小
dataset_train_size = len(dataset_train)  # 假设有 1000 个样本
total_step = 3 * dataset_train_size  # 3 个 epoch 的总步数

# 定义初始学习率和 warmup 比例
lr = 2e-5
warmup_ratio = 0.1

# 生成学习率列表
decay_lr = dynamic_lr(lr=lr, total_step=total_step, warmup_ratio=warmup_ratio)

# 打印前 10 个学习率
print("前 10 个学习率:", decay_lr[:10])
# 打印最后 10 个学习率
print("最后 10 个学习率:", decay_lr[-10:])

前 10 个学习率: [1.0582010582010582e-07, 2.1164021164021165e-07, 3.174603174603175e-07, 4.232804232804233e-07, 5.291005291005291e-07, 6.34920634920635e-07, 7.407407407407407e-07, 8.465608465608466e-07, 9.523809523809525e-07, 1.0582010582010582e-06]
最后 10 个学习率: [1.0582010582010694e-07, 9.406231628453776e-08, 8.230452674897083e-08, 7.054673721340388e-08, 5.8788947677836936e-08, 4.703115814226999e-08, 3.527336860670305e-08, 2.3515579071133888e-08, 1.1757789535566944e-08, 0.0]


In [15]:
from mindnlp.transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('tohoku-nlp/bert-base-japanese-v3', num_labels=2)

optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=decay_lr)


metric = Accuracy()

ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='bert_japanese_wrime_finetune', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='bert_japanese_wrime_finetune_best', auto_load=True)

trainer = Trainer(network=model, train_dataset=dataset_train,
                  eval_dataset=dataset_val, metrics=metric,
                  epochs=3, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb],
                  jit=False)

The following parameters in checkpoint files are not loaded:
['bert.embeddings.position_ids', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
The following parameters in models are missing parameter:
['classifier.weight', 'classifier.bias']


In [16]:
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0:   0%|          | 0/630 [00:00<?, ?it/s]

Epoch 0: 100%|█████████▉| 629/630 [03:45<00:00,  3.15it/s, loss=0.27174908]

Epoch 0: 100%|██████████| 630/630 [03:54<00:00,  2.68it/s, loss=0.27164856]


Checkpoint: 'bert_japanese_wrime_finetune_epoch_0.ckpt' has been saved in epoch: 0.


...

Evaluate Score: {'Accuracy': 0.9203980099502488}
---------------Best Model: 'bert_japanese_wrime_finetune_best.ckpt' has been saved in epoch: 1.---------------


Epoch 2: 100%|██████████| 630/630 [03:34<00:00,  2.94it/s, loss=0.045812417]


The maximum number of stored checkpoints has been reached.
Checkpoint: 'bert_japanese_wrime_finetune_epoch_2.ckpt' has been saved in epoch: 2.


Evaluate: 100%|██████████| 51/51 [00:07<00:00,  7.05it/s]


Evaluate Score: {'Accuracy': 0.9409203980099502}
---------------Best Model: 'bert_japanese_wrime_finetune_best.ckpt' has been saved in epoch: 2.---------------
Loading best model from 'checkpoint' with '['Accuracy']': [0.9409203980099502]...
---------------The model is already load the best model from 'bert_japanese_wrime_finetune_best.ckpt'.---------------


In [17]:
evaluator = Evaluator(network=model, eval_dataset=dataset_val, metrics=metric)
evaluator.run(tgt_columns="labels")

Evaluate: 100%|██████████| 51/51 [00:06<00:00,  7.33it/s]

Evaluate Score: {'Accuracy': 0.9409203980099502}
